# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego
Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego. 

### Autor rozwiązania
Uzupełnij poniższe informacje umieszczając swoje imię i nazwisko oraz numer indeksu:

In [ ]:
NAME = "Dominika Nowak"
ID = ""

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. **Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.**

Schemat oceniania:
- wynik na zbiorze Iris (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 10%: +40%,
- wynik na ukrytym zbiorze testowym 1 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 15%: +30%,
- wynik na ukrytym zbiorze testowym 2 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 5%: +30%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn). 
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [ ]:
import numpy as np

class TreeNode:
	def __init__(self):
		self.left: TreeNode | None = None  # prawda
		self.right: TreeNode | None = None  # niespelniony warunek
		self.splitFeature: int | None = None
		self.threshold: float | None = None
		self.prediction: int | None = None  # wartość liścia -> klasyfikacji

	@staticmethod
	def giniIndex(y): # w scikit-learn domyslny
		"""
		Args:
			y (np.ndarray): Wektor klas.
		Returns:
            float: Wartość indeksu Giniego.
		"""
		# gi = 1 - sum(p(i)^2)
		_, counts = np.unique(y, return_counts=True)
		probs = counts / len(y)
		return np.sum(probs**2)
	
	@staticmethod
	def entropy(y):
		_, counts = np.unique(y, return_counts=True)
		probs = counts / len(y)
		return -np.sum(probs * np.log2(probs))
	
	def bestSplit(self, data, target):
		bestFeature = None
		bestTreshold = None
		bestScore = -np.inf #inf bo maksymalizujemy information Gain, gdyby wykorzytstać Gini Index to +inf
		
		for feature in range(data.shape[1]):		# dla kazdej cechy
			tresholds = np.unique(data[:, feature]) # wszytskie mozliwe podzialy

			for i in range(1, len(tresholds)):
				treshold = (tresholds[i - 1] + tresholds[i]) / 2 #podział miedzy punktami danych
				leftMask = data[:, feature] <= treshold # True jesli true dla warunku
				rightMask = ~leftMask
			
				if not np.any(leftMask) or not np.any(rightMask):
					continue #pomijamy podziały dajace nam pusta grupe

				#leftScore = self.giniIndex(target[leftMask])
				#rightScore = self.giniIndex(target[rightMask])
				
				# Jednak zdecydowałam się na entropię, poniewaz daje ona lepsze wyniki na dwóch przetestowanych datasetach
				leftScore = self.entropy(target[leftMask])
				rightScore = self.entropy(target[rightMask])

				# cost function -> wazony gini
				#score = (leftScore * np.sum(leftMask) + rightScore * np.sum(rightMask))

				entropy = self.entropy(target)
				infoGain = entropy - ((np.sum(leftMask) / len(target)) * leftScore + (np.sum(rightMask) / len(target)) * rightScore)


				if infoGain > bestScore:
					bestScore = infoGain
					bestFeature = feature
					bestTreshold = treshold
		
		return bestFeature, bestTreshold




	def fit(self, data: np.ndarray, target: np.ndarray) -> None:
		"""
		Args:
			data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
			target (np.ndarray): wektor klas o długości n, gdzie n to liczba przykładów
		"""
		# Warunek zatrzymania -> jeśli wezel zawiera tylko jedną klase to przypisz ja
		# Możnaby dodać np. maksymalną głębokość drzewa, jednak nie wiem na jakich danych będzie wykorzystywana funkcja i nie mam też jeszcze wyczucia na jaką wartość by to można było ustawić
		if np.all(target == target[0]):
			self.prediction = target[0]
			return

		# Najlepszy podział
		self.splitFeature, self.threshold = self.bestSplit(data, target)

		if self.splitFeature is None:
			self.prediction = np.bincount(target).argmax()

		leftMask = data[:, self.splitFeature] <= self.threshold 
		rightMask = ~leftMask

		self.left = TreeNode()
		self.left.fit(data[leftMask], target[leftMask])

		self.right = TreeNode()
		self.right.fit(data[rightMask], target[rightMask])
		
	
	def predict(self, data: np.ndarray) -> np.ndarray:
		"""
		Args:
			data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

		Returns:
			np.ndarray: wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów
		"""
		y_pred = np.zeros(data.shape[0])

		for i, observation in enumerate(data):
			node = self

			while node.prediction is None:
				node = node.left if observation[node.splitFeature] <= node.threshold else node.right

			y_pred[i] = node.prediction

		return y_pred


## Przykład trenowanie i testowania drzewa
 
Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.88


### Wykorzytanie entropii dla load_digits

In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.867003367003367


### Wykorzystanie Gini Index dla load_digits

Prawdopodobnie z wykorzytaniem Gini Index zaszedł dla tego zbioru danych overfitting, trzeba by zaimplementować pruning. (Lub wykorzystać entropie, która bez pruningu daje okej wyniki na tych dwóch sprawdzonych datasetach.)

In [66]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_train)
print(accuracy_score(y_train, y_pred))
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

1.0
0.3434343434343434
